In [ ]:
#Adapting code for modifying state geojson file into Texas county geojson file. Use same strategy, modify
#the Texas county geojson properties directly without modification of the coordinates.

In [1]:
import pandas as pd
import numpy as np
import json
import os

In [2]:
ls

 Volume in drive C is OS
 Volume Serial Number is 3041-72E3

 Directory of C:\Users\StevenJBark\Desktop\RICEDAV\Projects\FinalProject_RiceBootcamp\SteveBranch\DataAnalytics-FinalProject\GeoJSON\Texas_Counties\2016_Texas_Counties_Files

03/27/2020  12:27 PM    <DIR>          .
03/27/2020  12:27 PM    <DIR>          ..
03/27/2020  12:27 PM    <DIR>          .ipynb_checkpoints
03/27/2020  11:29 AM           139,212 2016_TXCounty_Health_Data.csv
03/27/2020  12:21 PM            63,144 GeoJSON_TexasCounties_AddProperties.ipynb
03/21/2020  02:31 PM        16,167,870 texascounties.geojson
               3 File(s)     16,370,226 bytes
               3 Dir(s)  412,724,670,464 bytes free


# Extracting GeoJSON File for Texas Counties and Sorting

In [3]:
#Read into Pandas the texascounties.geojson file
txcounty_data = pd.read_json("./texascounties.geojson")

In [4]:
#Check the txcounty_data formatting to identify the features, properties that must be appended.
txcounty_data.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 8, '..."
1,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 36, ..."
2,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 12, ..."
3,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 71, ..."
4,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 14, ..."


In [5]:
#Checking names to see if order must be sorted. The order was not alphabetical, so sorted below.
# for line in txcounty_data["features"]:
#     print(line["properties"]["name"])

In [6]:
#Must sort the geojson by county name so I can generate the same order from the health care data by county.
#Create county_list, then append the county names in order to new list, then crate new column in the dataframe 
#for sorting by county name.
county_list = []
for line in txcounty_data["features"]:
    county_list.append(line["properties"]["name"])

#county_list  displayed all of the counties as expected in the new list.

#Create new column using the county_list
txcounty_data["county"] = county_list

#Check new txcounty_data dataframe
txcounty_data.head(10)

,type,features,county
0,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 8, '...",Menard
1,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 36, ...",Calhoun
2,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 12, ...",Hale
3,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 71, ...",Kerr
4,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 14, ...",Armstrong
5,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 39, ...",Clay
6,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 56, ...",Orange
7,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 64, ...",Gonzales
8,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 69, ...",Fannin
9,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 123,...",Jim Hogg


In [7]:
#Sort dataframe based on county column
sorted_txcounty_data = txcounty_data.sort_values("county", ascending=True)
sorted_txcounty_data.head(10)

,type,features,county
243,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 3138...",Anderson
77,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 960,...",Andrews
25,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 298,...",Angelina
59,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 707,...",Aransas
155,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 2045...",Archer
4,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 14, ...",Armstrong
186,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 2355...",Atascosa
202,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 2597...",Austin
189,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 2330...",Bailey
16,FeatureCollection,"{'type': 'Feature', 'properties': {'gid': 208,...",Bandera


In [8]:
#Set index to county to prevent problems with numerical index values.
indexed_txcounty_data = sorted_txcounty_data.set_index("county")

In [9]:
indexed_txcounty_data.count()

type        254
features    254
dtype: int64

# Read CSV File for Health Data: Extract Data for Properties in County GeoJSON

In [10]:
#Read in the 2019_Full.csv so we can import the health data for each county.
#Use input to allow us to decide which file to use for data.
user_input = input("Which CSV file do you want to use for health data?: ")
health_data = pd.read_csv(user_input)

Which CSV file do you want to use for health data?: 2016_TXCounty_Health_Data.csv


In [11]:
health_data.head()

,FIPS,State,County,Year,Yrs_Potential_Life_Lost_per100K(YPLL),Percent_Fair/Poor_Health,Percent_Low_Birthweight_Births,Percent_Adult_Smokers,Percent_Adult_Obesity,"Healthy_Food_Access(0=Worst,10=Best)",...,Percent_Native_Hawaiian/Other_Pacific_Islander,Number_Hispanic,Percent_Hispanic,Number_Non-Hispanic_White,Percent_Non-Hispanic_White,Number_Not_Proficient_in_English,Percent_Not_Proficient_in_English,Percent_Female,Number_Pop_Living_in_Rural_Area,Percent_Rural
0,48001,Texas,Anderson,2016,10847.5,19.8,8.059632,17.7,33.4,5.3,...,0.128412,9814,17.030212,34530,59.919829,1305,2.371348,39.049404,39204,67.063533
1,48003,Texas,Andrews,2016,8469.4,18.1,8.315900,14.3,28.8,7.4,...,0.022887,9538,54.574584,7269,41.591806,1086,7.378219,48.932883,2440,16.502097
2,48005,Texas,Angelina,2016,8808.5,20.3,9.039291,18.5,33.6,5.3,...,0.056980,18602,21.198860,53925,61.452991,3655,4.502507,51.076923,37383,43.082366
3,48007,Texas,Aransas,2016,11772.2,18.5,9.145674,15.2,28.0,5.5,...,0.088099,6666,26.693897,16960,67.916066,594,2.619394,50.452507,6313,27.260558
4,48009,Texas,Archer,2016,6091.8,12.8,7.606679,13.8,29.9,6.9,...,0.068097,721,8.182953,7802,88.548405,146,1.717849,49.892180,8057,88.988292


In [12]:
#Use count to confirm number of counties in county_health_data equals number of counties in geojson file.
health_data["County"].count()

254

In [13]:
#NaN causes problems for geojson formatted data. To check, use following code and replace all NaN with Null
#county_health_data.isnull().any()
#county_health_data2 = county_health_data.replace(to_replace=np.nan, value="Null")

#Checked Raquel's data and all NaN values were resolved. No county_health_data2 dataframe is required.

In [14]:
#Create a list of dictionaries with the proper names and data. Use the code from states_coordinates workbook with
#modification for only the data in the sorted_health_data dataframe. The idea is that the sorted data will be in the
#proper order in the list of dictionaries for appending into the statesData dataframe.

In [15]:
health_list = []

for row in health_data.iterrows():
    health_dict = {"properties": {"Year": row[1]["Year"],
                                "FIPS": row[1]["FIPS"],
                                "County": row[1]["County"],
                                "Years_Potential_Life_Lost_Per_100K": row[1]["Yrs_Potential_Life_Lost_per100K(YPLL)"],
                                "Percent_Fair/Poor_Health": row[1]["Percent_Fair/Poor_Health"],
                                "Percent_Low_Birthweight": row[1]["Percent_Low_Birthweight_Births"],
                                "Percent_Adult_Smokers": row[1]["Percent_Adult_Smokers"],
                                "Percent_Adult_Obesity": row[1]["Percent_Adult_Obesity"],
                                "Healthy_Food_Access, 0-10, 10=Best": row[1]["Healthy_Food_Access(0=Worst,10=Best)"],
                                "Percent_Adults_Physical_Inactivity": row[1]["Percent_Adults_Physical_Inactivity"],
                                "Percent_Access_to_Excercise": row[1]["Percent_Access_to_Excercise_Opportunities"],
                                "Percent_Adults_Excessive_Drinking": row[1]["Percent_Adults_Excessive_Drinking"],
                                "Percent_Driving_Deaths_with_Alcohol": row[1]["Percent_Driving_Deaths_with_Alcohol_Involvement"],
                                "Chlamydia_Cases_Per_100K": row[1]["Chlamydia_Cases_per_100K"],
                                "Teen_Birth_Rate": row[1]["Teen_Birth_Rate"],
                                "Percent_Uninsured_Under65": row[1]["Percent_Ppl_Under65_w/o_Insurance"],
                                "Primary_Physicians_Per_100K": row[1]["PCP_per_100K"],
                                "Dentists_Per_100K": row[1]["Dentists_per_100K"],
                                "Mental_Health_Professionals_Per_100K": row[1]["MHP_per_100K"],
                                "Preventable_Hospitalizations_Per_100K Medicare": row[1]["Preventable_Hospitalization_Rate(per100K_Medicare_Enrollees)"],
                                "Graduation_Rate": row[1]["Graduation_Rate"],
                                "Percent_Adults_Post-Secondary_Education(25-44)": row[1]["Percent_Adults(25-44)_with_Some_Post-secondary_Ed"],
                                "Percent_People_16+_Unemployed_Looking": row[1]["Percent_Ppl(16+)_Unemployed_&_Looking_for_Work"],
                                "Percent_Childeren_Poverty": row[1]["Percent_Children(under18)_Living_in_Poverty"],
                                "Percent_Children_SingleParentHome": row[1]["Percent_Children_Living_in_Single_Parent_Household"],
                                "80th_Percentile_MHI": row[1]["80th_Percentile_MHI"],
                                "20th_Percentile_MHI": row[1]["20th_Percentile_MHI"],
                                "Associations_Per_10K": row[1]["Associations_per_10K"],
                                "Violent_Crimes_Per_100K": row[1]["Violent_Crimes_per_100K"],
                                "Injury_Death_Per_100K": row[1]["Injury_Death_Rate_per_100K"],
                                "Avg_Daily_Particulates(microgram/meter-cubed)": row[1]["Avg_Daily_Particulate_Matter_in_Micrograms_per_Cubic_Meter"],
                                "Water_Violations": row[1]["County_Affected_by_Water_Violation(1=Yes,0=No)"],
                                "Percent_Homes_Cost_Plumbing_Crowding": row[1]["%_Households_with_One_or_More Problems(overcrowding,high-housing-cost,lack-kitchen,lack-plumbing-facilities)"],
                                "Lone_Commuter_Over_30Min": row[1]["Percent_Lone_Commuters_who_Commute_More_Than_30Min"],
                                "Premature_Mortality_Per_100K_Under75": row[1]["Premature(under75)_Age_Adjusted_Mortality_per_100K"],
                                "Premature_Mortality_Per_100K_Under18": row[1]["Child(under18)_Mortality_per_100K"],
                                "Infant_Mortality_Per_1K_Live_Births": row[1]["Infant(within1yr)_Mortality_per_1K_live_births"],
                                "Percent_Adults_Diabetes": row[1]["Percent_Adults(20+)_Diagnosed_Diabetes"],
                                "HIV_Per_100K_Over13": row[1]["Ppl(Age13+)_diagnosed_HIV_per_100K"],
                                "Percent_No_Adequate_Access_Food": row[1]["Percent_Pop_w/o_Adequate_Access_to_Food"],
                                "Drug_Overdose_Deaths_Per_100K": row[1]["Drug_Overdose_Deaths_per_100K"],
                                "Motor_Vehicle_Deaths_Per_100K": row[1]["Motor_Vehicle_Deaths_per_100K"],
                                "Percent_Children_No_Health_Insurance": row[1]["Percent_Children(under19)_w/o_Health_Insurance"],
                                "Other_Primary_Care_Providers_Non-Physician": row[1]["Other_Primary_Care_Providers(non_physician)_per_100K"],
                                "Median_Household_Income": row[1]["Median_Household_Income"],
                                "Homicides_Per_100K": row[1]["Homicide_per_100K"],
                                "Population": row[1]["Resident_Population"],
                                "Percent_Free_Reduced_Lunch": row[1]["Percent_Children_Enrolled_in_Free/Reduced_Lunch"],
                                "Percent_Under_18": row[1]["Percent_under_18_years_of_age"],
                                "Percent_Over_65": row[1]["Percent_Age_65_and_over"],
                                "Percent_African_American": row[1]["Percent_Non-Hispanic_Black_or_African_American"],
                                "Percent_American_Indian_Alaskan": row[1]["Percent_American_Indian_or_Alaskan_Native"],
                                "Percent_Asian": row[1]["Percent_Asian"],
                                "Percent_Pacific_Islander_Hawaiian": row[1]["Percent_Native_Hawaiian/Other_Pacific_Islander"],
                                "PercentPercent_Hispanic": row[1]["Percent_Hispanic"],
                                "Percent_White": row[1]["Percent_Non-Hispanic_White"],
                                "Percent_Not_English_Proficient": row[1]["Percent_Not_Proficient_in_English"],
                                "Percent_Female": row[1]["Percent_Female"],  
                                "Percent_Rural": row[1]["Percent_Rural"]
                                }
                }
    
    health_list.append(health_dict)

In [16]:
len(health_list)

254

In [17]:
#Now every item in the list is a dictionary in the same order as the sorted_txcounty_data dataframe from the working geojson file.
#Look at the sorted_txcounty_data["features"][0]["properties"] to confirm Anderson. Check some others.
#Then replace "properties" dictionary with the dictionary from the health_list at the same index and review.

In [18]:
indexed_txcounty_data["features"][246]["properties"]

{'gid': 2386,
 'statefp': '48',
 'countyfp': '493',
 'countyns': '01384032',
 'geoid': '48493',
 'name': 'Wilson',
 'namelsad': 'Wilson County',
 'lsad': '06',
 'classfp': 'H1',
 'mtfcc': 'G4020',
 'csafp': None,
 'cbsafp': '41700',
 'metdivfp': None,
 'funcstat': 'A',
 'aland': 2081663799.0,
 'awater': 12110414.0,
 'intptlat': '+29.1738830',
 'intptlon': '-098.0867336'}

In [19]:
health_list[246]["properties"]

{'Year': 2016,
 'FIPS': 48493,
 'County': 'Wilson',
 'Years_Potential_Life_Lost_Per_100K': 5582.1,
 'Percent_Fair/Poor_Health': 16.1,
 'Percent_Low_Birthweight': 7.669172932,
 'Percent_Adult_Smokers': 13.4,
 'Percent_Adult_Obesity': 27.6,
 'Healthy_Food_Access, 0-10, 10=Best': 8.1,
 'Percent_Adults_Physical_Inactivity': 28.0,
 'Percent_Access_to_Excercise': 58.83778368,
 'Percent_Adults_Excessive_Drinking': 17.9,
 'Percent_Driving_Deaths_with_Alcohol': 20.0,
 'Chlamydia_Cases_Per_100K': 265.9454586,
 'Teen_Birth_Rate': 30.33991947,
 'Percent_Uninsured_Under65': 19.78340847,
 'Primary_Physicians_Per_100K': 37.43009,
 'Dentists_Per_100K': 23.70587,
 'Mental_Health_Professionals_Per_100K': 21.5508,
 'Preventable_Hospitalizations_Per_100K Medicare': 52.1,
 'Graduation_Rate': 96.71939587,
 'Percent_Adults_Post-Secondary_Education(25-44)': 58.306999899999994,
 'Percent_People_16+_Unemployed_Looking': 4.1203948530000005,
 'Percent_Childeren_Poverty': 15.1,
 'Percent_Children_SingleParentHome'

In [20]:
for i in range(len(health_list)):
    indexed_txcounty_data["features"][i]["properties"] = health_list[i]["properties"]

In [21]:
indexed_txcounty_data

,type,features
county,,
Anderson,FeatureCollection,"{'type': 'Feature', 'properties': {'Year': 201..."
Andrews,FeatureCollection,"{'type': 'Feature', 'properties': {'Year': 201..."
Angelina,FeatureCollection,"{'type': 'Feature', 'properties': {'Year': 201..."
Aransas,FeatureCollection,"{'type': 'Feature', 'properties': {'Year': 201..."
Archer,FeatureCollection,"{'type': 'Feature', 'properties': {'Year': 201..."
Armstrong,FeatureCollection,"{'type': 'Feature', 'properties': {'Year': 201..."
Atascosa,FeatureCollection,"{'type': 'Feature', 'properties': {'Year': 201..."
Austin,FeatureCollection,"{'type': 'Feature', 'properties': {'Year': 201..."
Bailey,FeatureCollection,"{'type': 'Feature', 'properties': {'Year': 201..."


In [22]:
# Convert dataframe to list of dictionaries based on features.
data_list = []

for row in indexed_txcounty_data.iterrows():
    data_list.append(row[1][1])

In [23]:
#Append the data_list into a final dictionary to generate the complete geojson file.
final_dict = {"type": "FeatureCollection", "features": data_list}

In [24]:
#Writing out the list of dictionaries to a geojson file
with open("2016_TXCounty_Health_Data.geojson","w") as file:
    json.dump(final_dict, file)